In [1]:
!pip install statsmodels plotly --quiet

You should consider upgrading via the '/Users/danielmiller/.pyenv/versions/3.9.15/envs/torch_rl/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
USE_GPU = False
from typing import List, Tuple, Dict, Any, Optional
import seaborn as sns
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import re
import plotly.express as px

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt
import os
import glob
from typing import Dict
from statsmodels.tsa.filters.hp_filter import hpfilter
import re
from datetime import datetime

In [4]:
CORE_PATH = 'tensorboard_logs'
LOG_DIR = 'rl_stats/question_1'

In [5]:
n_files = 30
feature_ext = 'cnn'
lstm = 'label'
exec_time = 'ii'
base_exp_path = '/'.join(
        [
            'rl_experiments',
            f'n_files_{n_files}',
            f'{feature_ext}_{lstm}',
            exec_time,
        ]
    )
base_exp_path

'rl_experiments/n_files_30/cnn_label/ii'

In [6]:
def tensorboard_results(log_matrix: Dict, scalar_key: str):
    
    log_df = {}
    for label, log_dir in log_matrix.items():
        print(f'Getting {label} results')
        events = EventAccumulator(log_dir)
        events.Reload()
        reward = events.Scalars(scalar_key)
        df = pd.DataFrame({
                'wall_time': [x.wall_time for x in reward], # 'step': [x.step for x in loss],
                'step': [x.step for x in reward],
                scalar_key: [x.value for x in reward],
            })
        df['wall_time'] = pd.to_datetime(df['wall_time'], unit='s')
        log_df[label] = df
        file_prefix = re.sub(r'[^a-zA-Z0-9]', '_', label).lower()
        scalar_key_prefix = re.sub(r'[^a-zA-Z0-9]', '_', scalar_key).lower()
        file_path = os.path.join(LOG_DIR, f'{file_prefix}_{scalar_key_prefix}.csv')
        print(f'Writing to {file_path}')
        df.to_csv(file_path, index=False)
        
    return log_df

def concatenate_vectors(log_matrix, scalar):
    
    df_container = {}
    for k, v in log_matrix.items():
        k = re.sub(r'[^a-zA-Z0-9]', '_', k).lower()
        scalar_name = re.sub(r'[^a-zA-Z0-9]', '_', scalar).lower()
        df_container[k] = v[scalar].values
        
    min_shape = (
        min([v.shape[0] for v in df_container.values()]),
    )
    
    df = pd.DataFrame({
        k: v[:min_shape[0]] for k, v in df_container.items()

    })
    
    df['step'] = np.arange(min_shape[0])
    
    df.to_csv(os.path.join(LOG_DIR, f'{scalar_name}_concatenated.csv'), index=False)
    return df


def plot_vectors(df, y_reassign):
    df = df.copy() \
        .melt(id_vars=['step'], var_name='model', value_name=y_reassign) \
        .reset_index(drop=True)
    
    fig = px.line(
        df,
        x='step',
        y=y_reassign,
        color='model',
        title=f'{y_reassign} over steps for each model',
    )
    
    fig.show()

In [11]:
log_dirs_q1_dqn = {
    'DQN LABEL MLP': 'mlp-label.out.tfevents',
    'DQN NONE MLP': 'mlp-none.out.tfevents',
    'DQN PRED MLP': 'mlp-pred.out.tfevents',
   
    'DQN NONE CNN': 'cnn-none.out.tfevents', 
    'DQN PRED CNN': 'cnn-pred.out.tfevents',
}

log_dirs_q1 = log_dirs_q1_dqn.copy()
log_dirs_q1 = {
    k: os.path.join(CORE_PATH, v) for k, v in log_dirs_q1.items()
}


results_dir = os.path.join('rl_stats', 'q_1_cnn')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [12]:
# df_summary_stats = fetch_session_stats(log_dirs_q1)
rewards = tensorboard_results(log_dirs_q1, 'rollout/ep_rew_mean')
reward = concatenate_vectors(rewards, 'rollout/ep_rew_mean')
plot_vectors(reward, 'Mean Episode Reward')


Getting DQN LABEL MLP results
Writing to rl_stats/question_1/dqn_label_mlp_rollout_ep_rew_mean.csv
Getting DQN NONE MLP results
Writing to rl_stats/question_1/dqn_none_mlp_rollout_ep_rew_mean.csv
Getting DQN PRED MLP results
Writing to rl_stats/question_1/dqn_pred_mlp_rollout_ep_rew_mean.csv
Getting DQN NONE CNN results
Writing to rl_stats/question_1/dqn_none_cnn_rollout_ep_rew_mean.csv
Getting DQN PRED CNN results
Writing to rl_stats/question_1/dqn_pred_cnn_rollout_ep_rew_mean.csv


In [13]:
losses = tensorboard_results(log_dirs_q1, 'train/loss')
loss = concatenate_vectors(losses, 'train/loss')
plot_vectors(loss, 'Mean Training Loss')

Getting DQN LABEL MLP results
Writing to rl_stats/question_1/dqn_label_mlp_train_loss.csv
Getting DQN NONE MLP results
Writing to rl_stats/question_1/dqn_none_mlp_train_loss.csv
Getting DQN PRED MLP results
Writing to rl_stats/question_1/dqn_pred_mlp_train_loss.csv
Getting DQN NONE CNN results
Writing to rl_stats/question_1/dqn_none_cnn_train_loss.csv
Getting DQN PRED CNN results
Writing to rl_stats/question_1/dqn_pred_cnn_train_loss.csv
